In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [15]:
# model class 정의

class DropoutNet(nn.Module):
    def __init__(self):
        super(DropoutNet, self).__init__()
        
        self.fc1 = nn.Linear(784, 500)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

class NonDropoutNet(nn.Module):
    def __init__(self):
        super(NonDropoutNet, self).__init__()
        
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
# data set, data loader 준비

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.3, ))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.3, ))
])

train_dataset = datasets.MNIST(root="./data", train=True, download=False, transform=train_transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=False, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# model, criterion, optimizer 준비

dropout_model = DropoutNet()
nondropout_model = NonDropoutNet()

criterion = nn.CrossEntropyLoss()

dropout_optimizer = optim.SGD(dropout_model.parameters(), lr=0.001)
nondropout_optimizer = optim.SGD(nondropout_model.parameters(), lr=0.001)

# Training, Evaluation

for model, optimizer in [(dropout_model, dropout_optimizer), (nondropout_model, nondropout_optimizer)]:
    # Training

    for epoch in range(10):
        print(f"\n----------------Training, Epoch # {epoch}----------------\n")
        
        model.train()
        
        for images, labels in train_loader:
            optimizer.zero_grad()
            
            output = model(images)
            
            loss = criterion(output, labels)
            
            loss.backward()
            
            optimizer.step()
            
    # Evaluation

    print("\n----------------Evaluation----------------\n")
    
    model.eval()

    with torch.no_grad():
        correct = 0
        total = 0
        
        for images, labels in test_loader:
            output = model(images)
            
            _, pred = torch.max(output.data, 1)
            
            total += images.size(0)
            
            correct += (pred == labels).sum().item()
            
        print(f"Model: {'Dropout' if model == dropout_model else 'Non Dropout'}")
        print(f"Accuracy: {100 * (correct / total) :.2f}%")


----------------Training, Epoch # 0----------------


----------------Training, Epoch # 1----------------


----------------Training, Epoch # 2----------------


----------------Training, Epoch # 3----------------


----------------Training, Epoch # 4----------------


----------------Training, Epoch # 5----------------


----------------Training, Epoch # 6----------------


----------------Training, Epoch # 7----------------


----------------Training, Epoch # 8----------------


----------------Training, Epoch # 9----------------


----------------Evaluation----------------

Model: Dropout
Accuracy: 91.22%

----------------Training, Epoch # 0----------------


----------------Training, Epoch # 1----------------


----------------Training, Epoch # 2----------------


----------------Training, Epoch # 3----------------


----------------Training, Epoch # 4----------------


----------------Training, Epoch # 5----------------


----------------Training, Epoch # 6----------------


----